# PFS 

The support for PFS files have been extended with MIKE IO release 1.2. It was previously only possible to *read* PFS files. It is now also possible to *modify* and *create* new PFS files. 

In [1]:
import mikeio

## Read

In [2]:
pfs = mikeio.read_pfs("../tests/testdata/pfs/lake.sw")
pfs

<mikeio.Pfs>
FemEngineSW: [DOMAIN]   Touched = 1   discretization = 2  ...

The "target" (root section) is in this case called FemEngineSW. `pfs.FemEngineSW` is a PfsSection object that contains other PfsSection objects. Let's print the names of it's subsections:

In [3]:
pfs.FemEngineSW.keys()

dict_keys(['DOMAIN', 'TIME', 'MODULE_SELECTION', 'SPECTRAL_WAVE_MODULE'])

It is possible to navigate to each section and keyword in the pfs file:

In [4]:
pfs.FemEngineSW.DOMAIN.file_name

'|.\\Lake_Mesh.mesh|'

In [ ]:
pfs.FemEngineSW.MODULE_SELECTION

In [ ]:
pfs.FemEngineSW.MODULE_SELECTION.mode_of_spectral_wave_module

If you are unsure the name of a section, it is also possible to search for a specific string in the file, to find the name of a specific section.

In the example below we do an case-insensitive search for the string 'charnock', which occurs at 6 different places in this file.

In [22]:
pfs.search("charnock")

[FemEngineSW]
   [SPECTRAL_WAVE_MODULE]
      [WIND]
         background_Charnock_parameter = 0.01
         Charnock_parameter = 0.01
      EndSect  // WIND
      [OUTPUTS]
         [OUTPUT_1]
            [MODEL_PARAMETERS]
               Charnock_constant = 0
            EndSect  // MODEL_PARAMETERS
         EndSect  // OUTPUT_1
         [OUTPUT_2]
            [MODEL_PARAMETERS]
               Charnock_constant = 0
            EndSect  // MODEL_PARAMETERS
         EndSect  // OUTPUT_2
         [OUTPUT_3]
            [MODEL_PARAMETERS]
               Charnock_constant = 0
            EndSect  // MODEL_PARAMETERS
         EndSect  // OUTPUT_3
         [OUTPUT_4]
            [MODEL_PARAMETERS]
               Charnock_constant = 0
            EndSect  // MODEL_PARAMETERS
         EndSect  // OUTPUT_4
      EndSect  // OUTPUTS
   EndSect  // SPECTRAL_WAVE_MODULE
EndSect  // FemEngineSW

The same search can be done at any level of the hierarchy, i.e. to search only within the OUTPUTS section:

In [25]:
pfs.FemEngineSW.SPECTRAL_WAVE_MODULE.OUTPUTS.search("charnock")

[OUTPUT_1]
   [MODEL_PARAMETERS]
      Charnock_constant = 0
   EndSect  // MODEL_PARAMETERS
EndSect  // OUTPUT_1
[OUTPUT_2]
   [MODEL_PARAMETERS]
      Charnock_constant = 0
   EndSect  // MODEL_PARAMETERS
EndSect  // OUTPUT_2
[OUTPUT_3]
   [MODEL_PARAMETERS]
      Charnock_constant = 0
   EndSect  // MODEL_PARAMETERS
EndSect  // OUTPUT_3
[OUTPUT_4]
   [MODEL_PARAMETERS]
      Charnock_constant = 0
   EndSect  // MODEL_PARAMETERS
EndSect  // OUTPUT_4

In [30]:
pfs.FemEngineSW.SPECTRAL_WAVE_MODULE.WIND.search("charnock")

background_Charnock_parameter = 0.01
Charnock_parameter = 0.01

MIKE FM PFS files has a specific structure and active FM modules can be accessed by an alias on the Pfs object. In this case, `pfs.SW` can be used as a short-hand for `pfs.FemEngineSW.SPECTRAL_WAVE_MODULE`.

In [ ]:
pfs.SW.SPECTRAL.number_of_directions

In [ ]:
pfs.SW.SPECTRAL.maximum_threshold_frequency

Enumerated sections (e.g. [OUTPUT_1], [OUTPUT_2], ...) can be outputted in tabular form (dataframe).

In [ ]:
df = pfs.SW.OUTPUTS.to_dataframe()
df

## Modify

The PfsSection object can be modified. Existing values can be changes, new key-value pairs can be added, subsections can added or removed. 

In [ ]:
pfs.SW.SPECTRAL.number_of_directions = 32

In [ ]:
pfs.SW.SPECTRAL

### Add a new keyword

In [ ]:
pfs.SW.SPECTRAL["new_keyword"] = "new_value"

In [29]:
pfs.SW.SPECTRAL

Touched = 1
type_of_frequency_discretization = 2
number_of_frequencies = 25
minimum_frequency = 0.055
frequency_interval = 0.02
frequency_factor = 1.1
type_of_directional_discretization = 1
number_of_directions = 32
minimum_direction = 0.0
maximum_direction = 0.0
separation_of_wind_sea_and_swell = 0
threshold_frequency = 0.125
maximum_threshold_frequency = 0.5959088268863615
new_keyword = 'new_value'

### Add a section

Let's create an additional output, by copying OUTPUT_4 and modifying some parameters.

In [30]:
pfs.SW.OUTPUTS.number_of_outputs += 1

In [31]:
new_output = pfs.SW.OUTPUTS.OUTPUT_4.copy()

In [32]:
new_output.file_name = 'spectrum_x10km_y40km.dfsu'
new_output.POINT_1.x = 10000
new_output.POINT_1.y = 40000

In [33]:
pfs.SW.OUTPUTS["OUTPUT_5"] = new_output

In [34]:
pfs.SW.OUTPUTS.keys()

dict_keys(['Touched', 'MzSEPfsListItemCount', 'number_of_outputs', 'OUTPUT_1', 'OUTPUT_2', 'OUTPUT_3', 'OUTPUT_4', 'OUTPUT_5'])

## Output

The Pfs object can be written to pfs file, but can also be exported to a dictionary (which in turn can be written to a yaml or json file).

In [ ]:
pfs.write("lake_modified.pfs")

In [ ]:
pfs.to_dict()

In [ ]:
# write to yaml file
import yaml
yaml.dump(pfs.to_dict(), open('lake_modified.yaml', 'w+'))

## Create

A PFS file can also be created from a dictionary, like this:

In [ ]:
setup = {
    "Name": "Extract that",
    "InputFileName": "|random.dfs1|",
    "FirstTimeStep": 0,
    "LastTimeStep": 99,
    "X": 2,
    "OutputFileName": "|.\\out2.dfs0|",
}
t1_t0 = {"CLSID": "t1_t0.dll", "TypeName": "t1_t0", "Setup": setup}
t1_t0

In [ ]:
section = mikeio.PfsSection(t1_t0)
section

In [ ]:
pfs = section.to_Pfs(name="t1_t0")
pfs

In [ ]:
pfs.write("extract_point.mzt")

## Clean up

In [ ]:
import os
os.remove("lake_modified.pfs")
os.remove('lake_modified.yaml')
os.remove("extract_point.mzt")